In [12]:
import sys
#!{sys.executable} -m pip install --upgrade pip
#!{sys.executable} -m pip install jieba --upgrade

import docx
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
import os
import pandas as pd
import jieba
import pickle

jieba.__version__
# doc = docx.Document("WZWR minus names.docx")

'0.42.1'

In [13]:
from collections import Counter
def Count_func(words):
    c = Counter()
    for x in words:
        c[x] += 1
    return(c)

In [14]:
docs = []
for x,y,filenames in os.walk('./'):
    for filename in filenames:        
        if os.path.splitext(filename)[1] == '.docx' and x != './English':
            doc = docx.Document(filename)
            all_paras = doc.paragraphs
            indp_chr = '' #document's word as independent characters
            for x in all_paras:
                for y in x.text:
                    indp_chr+= y + ' '
            docs.append(indp_chr)
tf_docs = []
for x in docs:
    tf = Count_func(x)
#     tf.drop(' ',None)
#     tf.drop('，',None)
    tf_docs.append(tf)
print(tf_docs)

[Counter({' ': 10365, '，': 929, '。': 417, '之': 355, '不': 264, '其': 233, '以': 230, '而': 215, '也': 169, '臣': 159, '；': 133, '則': 128, '者': 122, '為': 110, '主': 97, '人': 96, '故': 90, '於': 80, '下': 78, '君': 75, '所': 70, '有': 65, '國': 61, '事': 54, '無': 52, '上': 51, '言': 49, '大': 49, '此': 48, '使': 47, '王': 46, '矣': 44, '曰': 43, '能': 43, '是': 43, '天': 42, '與': 41, '、': 41, '法': 39, '民': 38, '一': 38, '用': 38, '可': 37, '：': 36, '見': 36, '謂': 35, '行': 35, '必': 35, '群': 35, '名': 34, '道': 34, '亡': 33, '非': 33, '秦': 31, '子': 31, '死': 30, '得': 30, '趙': 29, '荊': 27, '將': 27, '功': 27, '乃': 27, '明': 27, '自': 27, '刑': 27, '韓': 26, '賞': 26, '兵': 26, '當': 25, '罰': 25, '夫': 25, '魏': 24, '齊': 24, '強': 24, '舉': 24, '失': 24, '私': 24, '知': 23, '皆': 23, '同': 23, '去': 23, '公': 23, '「': 21, '智': 21, '」': 21, '令': 21, '賢': 21, '燕': 20, '從': 20, '百': 20, '地': 20, '外': 20, '說': 20, '三': 19, '數': 19, '至': 19, '利': 19, '內': 19, '威': 19, '今': 18, '戰': 18, '四': 18, '侯': 18, '欲': 18, '治': 17, '萬': 17, '貴': 17, '？': 17, '固

In [15]:
df = {}
for x in tf_docs:
    for y in x:
        if y not in df:
            df[y] = 0
        df[y] +=1

In [16]:
stopwords = []
for x in df:
    if df[x] >= 5000:
        stopwords.append(x)

In [17]:
stopwords
#with open('acar.pickle', 'wb') as handle:
#    pickle.dump(stopwords, handle, protocol=pickle.HIGHEST_PROTOCOL)

[]

In [18]:
cleaned_tf_docs = []
for tf in tf_docs:
    for cha in stopwords:
        tf.pop(cha,None)
    cleaned_tf_docs.append(tf)

In [19]:
import math

In [20]:
for index,tf in enumerate(cleaned_tf_docs):
    ssum = 0
    for x in tf.values():
        ssum += x
    for key in tf:
        tf[key] = tf[key]/ssum * math.log(11/df[key]) # tf-idf
        
    cleaned_tf_docs[index] = tf
cleaned_tf_docs[0]

Counter({'臣': -0.0006793090907076078,
         ' ': -0.04428326242254311,
         '聞': 0.0011718782918264733,
         '：': -0.00015380583185832628,
         '「': -8.972006858402368e-05,
         '不': -0.001127909433627726,
         '知': -9.826483702059734e-05,
         '而': -0.000918562606931671,
         '言': -0.0002093468266960552,
         '，': -0.0039690449387884755,
         '智': 0.00020691763746956575,
         '；': -0.0005682271010321499,
         '忠': -3.417907374629473e-05,
         '。': -0.0017815842190256129,
         '」': -8.972006858402368e-05,
         '為': 0.007017633715718782,
         '人': -0.00041014888495553674,
         '當': 0.002092639806832988,
         '死': 0.0,
         '亦': -8.544768436573682e-06,
         '雖': 0.0008293567118576741,
         '然': -6.408576327430262e-05,
         '願': 0.0004185279613665976,
         '悉': 6.254615164853866e-05,
         '所': -0.0002990668952800789,
         '唯': 0.0,
         '大': -0.0002093468266960552,
         '王': -0.00019

In [21]:
filenamess = []
for x,y,filenames in os.walk('./'):
    for filename in filenames:        
        if os.path.splitext(filename)[1] == '.docx':
            filenamess.append(filename)

In [22]:
from scipy import spatial

for i in range(11):
    features = []
    for x in cleaned_tf_docs:
        temp = []
        for word in cleaned_tf_docs[i]:
            if word not in x:
                temp.append(0)
            else:
                temp.append(x[word])
        features.append(temp)
    for index,x in enumerate(features):
        result = 1 - spatial.distance.euclidean(features[i],features[index])
        print(filenamess[i],result,filenamess[index])
    print('===')
    

HFZ1.docx 1.0 HFZ1.docx
HFZ1.docx 0.9900798503642076 HFZ2.docx
HFZ1.docx 0.9829663567769815 ZCG_Chu.docx
HFZ1.docx 0.9829315693541156 ZCG_Han.docx
HFZ1.docx 0.9830909748709858 ZCG_Qi.docx
HFZ1.docx 0.9829663695083511 ZGC_Chu.docx
HFZ1.docx 0.9829314810377339 ZGC_Han.docx
HFZ1.docx 0.9830830215444044 ZGC_Qin.docx
HFZ1.docx 0.9795372725321143 ZGC_Speaking.docx
HFZ1.docx 0.9830507789715675 ZGC_Wei.docx
HFZ1.docx 0.9831153524302887 ZGC_Yan.docx
HFZ1.docx 0.983058891029729 ZGC_Zhao.docx
===
HFZ2.docx 0.9906255938966437 HFZ1.docx
HFZ2.docx 1.0 HFZ2.docx
HFZ2.docx 0.9841763050624188 ZCG_Chu.docx
HFZ2.docx 0.9840943402690777 ZCG_Han.docx
HFZ2.docx 0.9841978605126565 ZCG_Qi.docx
HFZ2.docx 0.984176318767285 ZGC_Chu.docx
HFZ2.docx 0.9840943539200395 ZGC_Han.docx
HFZ2.docx 0.9841884709707107 ZGC_Qin.docx
HFZ2.docx 0.9778743207362723 ZGC_Speaking.docx
HFZ2.docx 0.9841857386479546 ZGC_Wei.docx
HFZ2.docx 0.9842287935730594 ZGC_Yan.docx
HFZ2.docx 0.9841840730024908 ZGC_Zhao.docx
===
ZCG_Chu.docx 0.995

In [12]:
from scipy import spatial

for i in range(11):
    features = []
    for index,x in enumerate(cleaned_tf_docs):
        a = set(cleaned_tf_docs[i].keys())
        b = set(x.keys())
        c = a.intersection(b)
        print(filenamess[i],float(len(c)) / (len(a) + len(b) - len(c)),filenamess[index])

Speaking to the King in Zheng 722.docx 1.0 Speaking to the King in Zheng 722.docx
Speaking to the King in Zheng 722.docx 0.1483606557377049 Xunzi 10k.docx
Xunzi 10k.docx 0.1483606557377049 Speaking to the King in Zheng 722.docx
Xunzi 10k.docx 1.0 Xunzi 10k.docx


IndexError: list index out of range

In [14]:
#import the TfidfVectorizer from Scikit-Learn.
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=.65, min_df=1, stop_words=None, use_idf=True, norm=None)
transformed_docs = vectorizer.fit_transform(cleaned_tf_docs)

AttributeError: 'Counter' object has no attribute 'lower'